# Here we make a first pass at an attempt to use PCA in order to make our variables orthogonal. I think this could be an important step as we have many highly correlated variables such as pts_per_game, pts_per_poss, and pts_per_mp. PCA will allow us to take full advantage of these variables while removing redundancy

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, mutual_info_regression, f_regression
from sklearn.ensemble import RandomForestRegressor
import numpy as np

from sklearn.decomposition import PCA

pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import logging
from sklearn import tree
from sklearn.preprocessing import PolynomialFeatures, normalize, StandardScaler, MinMaxScaler
from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor, AdaBoostRegressor

from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle

In [15]:
dataTrain= pd.read_csv('../Data Scraping/DataTrainSet.csv')
dataVal = pd.read_csv('../Data Scraping/DataValidateSet.csv')
dataTest = pd.read_csv('../Data Scraping/DataTestSet.csv')

features = ['player','pos_x','age_x','team_id_x','g_x','mp_x','fg_per_poss','fga_per_poss','fg_pct_x'
              ,'fg3_per_poss','fg3a_per_poss','fg3_pct_x','fg2_per_poss','fg2a_per_poss','fg2_pct_x','ft_per_poss'
              ,'fta_per_poss','ft_pct_x','orb_per_poss','drb_per_poss','trb_per_poss','ast_per_poss','stl_per_poss'
              ,'blk_per_poss','tov_per_poss','pf_per_poss','pts_per_poss','off_rtg','def_rtg','per','ts_pct'
              ,'fg3a_per_fga_pct','fta_per_fga_pct','orb_pct','drb_pct','trb_pct','ast_pct','stl_pct','blk_pct'
              ,'tov_pct','usg_pct','ows','dws','ws','ws_per_48','obpm','dbpm','bpm','vorp'
              ,'mp_per_g','fg_per_g','fga_per_g','fg3_per_g','fg3a_per_g'
              ,'fg2_per_g','fg2a_per_g','efg_pct','ft_per_g','fta_per_g','orb_per_g','drb_per_g','trb_per_g'
              ,'ast_per_g','stl_per_g','blk_per_g','tov_per_g','pf_per_g','pts_per_g','fg_per_mp','fga_per_mp'
              ,'fg3_per_mp','fg3a_per_mp','fg2_per_mp','fg2a_per_mp','ft_per_mp','fta_per_mp','orb_per_mp'
              ,'drb_per_mp','trb_per_mp','ast_per_mp','stl_per_mp','blk_per_mp','tov_per_mp','pf_per_mp','pts_per_mp'
              ,'votes_first','points_won']

feat = features[4:-2]
XTrainDF = dataTrain[feat]
XValDF = dataVal[feat]
XTestDF = dataTest[feat]

XTrainDF = XTrainDF.fillna(0)
XValDF = XValDF.fillna(0)
XTestDF = XTestDF.fillna(0)

XTrain = XTrainDF.to_numpy()
XVal = XValDF.to_numpy()
XTest = XTestDF.to_numpy()

yTrain = dataTrain['points_won'].to_numpy()
yVal = dataVal['points_won'].to_numpy()


In [16]:
pca = PCA(n_components = 20)
pca.fit(XTrain)

PCA(copy=True, iterated_power='auto', n_components=20, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [17]:
XTrain_pca = pca.transform(XTrain)
XVal_pca = pca.transform(XVal)
XTest_pca = pca.transform(XTest)

In [18]:
features = ['ws', 'pts_per_g', 'vorp','fta_per_g', 'bpm', 'fg2_per_g','ows','ows','per','mp_per_g']


# features = ['g_x','mp_x','fg_per_poss','fga_per_poss','fg_pct_x'
#               ,'fg3_per_poss','fg3a_per_poss','fg3_pct_x','fg2_per_poss','fg2a_per_poss','fg2_pct_x','ft_per_poss'
#               ,'fta_per_poss','ft_pct_x','orb_per_poss','drb_per_poss','trb_per_poss','ast_per_poss','stl_per_poss'
#               ,'blk_per_poss','tov_per_poss','pf_per_poss','pts_per_poss','off_rtg','def_rtg','per','ts_pct'
#               ,'fg3a_per_fga_pct','fta_per_fga_pct','orb_pct','drb_pct','trb_pct','ast_pct','stl_pct','blk_pct'
#               ,'tov_pct','usg_pct','ows','dws','ws','ws_per_48','obpm','dbpm','bpm','vorp'
#               ,'mp_per_g','fg_per_g','fga_per_g','fg3_per_g','fg3a_per_g'
#               ,'fg2_per_g','fg2a_per_g','efg_pct','ft_per_g','fta_per_g','orb_per_g','drb_per_g','trb_per_g'
#               ,'ast_per_g','stl_per_g','blk_per_g','tov_per_g','pf_per_g','pts_per_g','fg_per_mp','fga_per_mp'
#               ,'fg3_per_mp','fg3a_per_mp','fg2_per_mp','fg2a_per_mp','ft_per_mp','fta_per_mp','orb_per_mp'
#               ,'drb_per_mp','trb_per_mp','ast_per_mp','stl_per_mp','blk_per_mp','tov_per_mp','pf_per_mp','pts_per_mp']

XTrainDF = dataTrain[features]
XValDF = dataVal[features]
XTestDF = dataTest[features]

XTrainDF = XTrainDF.fillna(0)
XValDF = XValDF.fillna(0)
XTestDF = XTestDF.fillna(0)

XTrain = XTrainDF.to_numpy()
XVal = XValDF.to_numpy()
XTest = XTestDF.to_numpy()

In [20]:
def pipeline(data_frame, estimators, params, filename, poly_fit):

    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
    logging.basicConfig(filename=filename, filemode='w', level=logging.INFO)
    logger = logging.getLogger()
    
    minimal_error, best_estimator = None, None
    
    for estimator in estimators:
        
        try: 
            print(f"Starting with estimator: {estimator.__name__}")
            logging.info(f"Starting with estimator: {estimator.__name__}")

            for index, cur_params in enumerate(params[estimator.__name__]):
                print(cur_params)
                regressor = estimator(**cur_params)

                # To collect MSE over each split
                errors = []

                # to collect accuracies
                accuracies = []
                top_1_accs = []

                #validation_data = data_frame
                #validation_data = validation_data.sort_values(by='points_won', ascending=False)

                # Get train data
                train_x = XTrain
                train_y = yTrain
                
                # Validate over one season only
                val_x = XVal
                val_y = yVal
                #val_y = val_y.reshape(val_y.shape[0], )
                

                if poly_fit is not None:
                    train_x = poly_fit.fit_transform(train_x)
                    val_x = poly_fit.fit_transform(val_x)

                
                shuffle_x, shuffle_y = shuffle(train_x, train_y)
                

                regressor.fit(shuffle_x, shuffle_y)
                predicted_y = regressor.predict(val_x)

                sorted_indices = np.argsort(predicted_y)[::-1]
                correct_indices = np.arange(len(val_y))

                curr_error = mean_squared_error(val_y, predicted_y)
                errors.append(curr_error)
                mean_error = np.average(errors)
            
                logging.info(f"Params: {cur_params}, MSE over all splits is: {mean_error:.4f}")
                print(
                    f"Params: {cur_params}, MSE over all splits is: {mean_error:.4f}")

                if minimal_error is None or mean_error < minimal_error:
                    minimal_error = mean_error
                    best_estimator = estimator(**cur_params)
        except Exception:
            print(f"Exception: {estimator}")
            continue
            
    return best_estimator

def pipelinePCA(data_frame, estimators, params, filename, poly_fit):

    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
    logging.basicConfig(filename=filename, filemode='w', level=logging.INFO)
    logger = logging.getLogger()
    
    minimal_error, best_estimator = None, None
    
    for estimator in estimators:
        
        try: 
            print(f"Starting with estimator: {estimator.__name__}")
            logging.info(f"Starting with estimator: {estimator.__name__}")

            for index, cur_params in enumerate(params[estimator.__name__]):
                print(cur_params)
                regressor = estimator(**cur_params)

                # To collect MSE over each split
                errors = []

                # to collect accuracies
                accuracies = []
                top_1_accs = []

                #validation_data = data_frame
                #validation_data = validation_data.sort_values(by='points_won', ascending=False)

                # Get train data
                train_x = XTrain_pca
                train_y = yTrain
                
                # Validate over one season only
                val_x = XVal_pca
                val_y = yVal
                #val_y = val_y.reshape(val_y.shape[0], )
                

                if poly_fit is not None:
                    train_x = poly_fit.fit_transform(train_x)
                    val_x = poly_fit.fit_transform(val_x)

                
                shuffle_x, shuffle_y = shuffle(train_x, train_y)
                

                regressor.fit(shuffle_x, shuffle_y)
                predicted_y = regressor.predict(val_x)

                sorted_indices = np.argsort(predicted_y)[::-1]
                correct_indices = np.arange(len(val_y))

                curr_error = mean_squared_error(val_y, predicted_y)
                errors.append(curr_error)
                mean_error = np.average(errors)
            
                logging.info(f"Params: {cur_params}, MSE over all splits is: {mean_error:.4f}")
                print(
                    f"Params: {cur_params}, MSE over all splits is: {mean_error:.4f}")

                if minimal_error is None or mean_error < minimal_error:
                    minimal_error = mean_error
                    best_estimator = estimator(**cur_params)
        except Exception:
            print(f"Exception: {estimator}")
            continue
            
    return best_estimator

In [21]:
estimators = [Ridge]

params = {Ridge.__name__: 
    [
        {
            'alpha': 1.0, 'solver':'svd'
        },
        {
            'alpha': 50.0, 'solver':'svd'
        },
    ],
         }

estimators = [Ridge]

In [25]:
best_estimator = pipeline(
    data_frame= None,
    estimators=estimators,
    params=params,
    filename="log_reg_poly_2.txt",
    # scaler=MinMaxScaler(),
    poly_fit=PolynomialFeatures(degree=3, interaction_only=True),
)

Starting with estimator: Ridge
{'alpha': 1.0, 'solver': 'svd'}
Params: {'alpha': 1.0, 'solver': 'svd'}, MSE over all splits is: 0.0011
{'alpha': 50.0, 'solver': 'svd'}
Params: {'alpha': 50.0, 'solver': 'svd'}, MSE over all splits is: 0.0013


In [24]:
best_estimator = pipelinePCA(
    data_frame= None,
    estimators=estimators,
    params=params,
    filename="log_reg_poly_2.txt",
    # scaler=MinMaxScaler(),
    poly_fit=PolynomialFeatures(degree=3, interaction_only=True),
)

Starting with estimator: Ridge
{'alpha': 1.0, 'solver': 'svd'}
Params: {'alpha': 1.0, 'solver': 'svd'}, MSE over all splits is: 0.0017
{'alpha': 50.0, 'solver': 'svd'}
Params: {'alpha': 50.0, 'solver': 'svd'}, MSE over all splits is: 0.0015
